<a href="https://colab.research.google.com/github/light-c0d3/CP468_Project/blob/main/CP468_Final_Project-Group13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CP468 Project

Tanya Yousofi,
Ruveyda Nur Kizmaz,
Zahra Mohamed,
Malika Sharma


All necessary pip installments

In [ ]:
%%capture
# code here for pip installments
!pip install scikit-learn pandas tensorflow keras matplotlib numpy gradio pillow opencv-python

All necessary libraries

In [ ]:
# code here for libraries
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf
from keras.applications import vgg16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np
import cv2
import random
import os
import gradio as gr
import traceback
from keras.layers import Dropout
from keras import regularizers
from keras.models import load_model
from sklearn.utils import shuffle
from scipy.stats import mode
from tensorflow.keras.applications.vgg16 import preprocess_input


Mount to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Phase 1 - CNN Model from scratch

In [ ]:
print("Our files:")
!ls /content/drive/MyDrive/KaggleDataset

zip_file_path1 = "/content/drive/MyDrive/KaggleDataset/archive.zip"
zip_file_path2 = "/content/drive/MyDrive/KaggleDataset/archive2.zip"

# Create directories if they do not exist:
!mkdir -p /content/KaggleDataset/archive
!mkdir -p /content/KaggleDataset/archive2

# Add your own path files here: + unzip
!unzip -o $zip_file_path1 -d /content/KaggleDataset
!unzip -o $zip_file_path2 -d /content/KaggleDataset

# Our data within archive and archive2
!ls /content/KaggleDataset/archive/
!ls /content/KaggleDataset/archive2/

# we will save these paths to not type the path each time
path1 = "/content/KaggleDataset/archive"
path2 = "/content/KaggleDataset/archive2"

In [ ]:
# take a look at the header of demographics table and store the file in demographic
# note the demographics and other tables are in part1 path
demographic = pd.read_csv(f"{path1}/demographic.csv")
print(demographic.head())

# Create dictionary to map subject IDs to respective groups healthy or schizo
# Load demographic data
# format = patient# : 0 /1 [0 = helthy and 1= schizophrenia]
diagnosis_dict = dict(zip(demographic.subject, demographic[" group"]))
print("Patient# : 0 / 1\n", diagnosis_dict)

# Load column labels which correspond to EEG electrodes
column_list = pd.read_csv(f"{path1}/columnLabels.csv").columns
all_elect = list(column_list[4:])
print("\nElectrodes List:\n",all_elect)

# See how many electrodes are there in total
print("\nTotal of electrodes used:\n", len(all_elect))  # 70


In [ ]:
# Initialize arrays for EEG data and labels
N_AVERAGED = 16
# x_counter = 0
EXPECTED_TRIALS = 100  #  100 trials per patient [change depending on data]
max_trials = 81 * EXPECTED_TRIALS  # 81 patients * 100 trials (adjust as needed)
input_size = 9216 * len(all_elect) // N_AVERAGED  # Size after averaging

X = np.zeros((max_trials, input_size), dtype="float32")
Y = np.zeros(max_trials)

print("Input size:\n", input_size)

# Function to average rows and reduce dimensionality
def avg_rows(a, n):
    shape = a.shape
    assert len(shape) == 2
    assert shape[0] % n == 0
    b = a.reshape(shape[0] // n, n, shape[1])
    mean_vec = b.mean(axis=1)
    return mean_vec

In [ ]:
# Process each patient's EEG data
x_counter = 0
for patient_num in range(1, 81 + 1):
    # Check if the path points to a directory
    # this part is important in case use of different path files
    person_dir_path = os.path.join(path1, f"{patient_num}.csv")
    if os.path.isdir(person_dir_path):
        csv_path = os.path.join(person_dir_path, f"{patient_num}.csv")
    else:
        csv_path = os.path.join(path2, f"{patient_num}.csv")
        if os.path.isdir(csv_path):
            csv_path = os.path.join(csv_path, f"{patient_num}.csv")
    try:
        df = pd.read_csv(csv_path, header=None, names=column_list)
    except FileNotFoundError:
        print(f"File not found for person number {patient_num} at {csv_path}")
        continue
    except IsADirectoryError:
        print(f"Path is a directory, not a file: {csv_path}")
        continue

    trials_list = set(df.trial)
    valid_trials_count = 0  # To count valid trials for each subject

    for trial_number in trials_list:
        number_of_trials = len(df[df.trial == trial_number])
        if number_of_trials == 9216.0:
            valid_trials_count += 1
            matrix = df[df.trial == trial_number][all_elect].values
            avg_n = avg_rows(matrix, n=N_AVERAGED)
            avg_n_vect = avg_n.reshape(-1)
            # to see the test for each patient un comment those otherwise it gives a long output
            # print(f"x_counter: {x_counter}, size of X: {X.shape[0]}")
            # print(f"Shape of avg_n_vect: {avg_n_vect.shape}")

            if x_counter < len(X):
                X[x_counter] = avg_n_vect.astype(np.float32)
                Y[x_counter] = diagnosis_dict[patient_num]
                x_counter += 1
            else:
                print(f"Reached limit for array counter {x_counter}")
                break

    print(f"Valid Trials: {valid_trials_count}  ::  For patient number: {patient_num}")


In [ ]:
# final number of valid trials
print(f"total valid trials: {x_counter}")

# trim unused portion of arrays
X = X[:x_counter]
Y = Y[:x_counter]

print(f"Final size of X: {X.shape} ") # (7092, 40320)
print(f"Final size of Y: {Y.shape}") # (7092,)

# Normalize the data
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Reshape data for CNN input
"""
7092 data points [labels]
each represented by a feature vector of size 40320 (9216 * 92)
"""
X_normalized = X_normalized.reshape((X_normalized.shape[0], X_normalized.shape[1], 1))
print(X.shape)
print(Y.shape)
# our input is ready for sequential model !

In [ ]:
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_normalized.shape[1], 1)),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    Dropout(0.5),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.25),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# Train the model & save the training history for plotting
history = model.fit(X_normalized, Y, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_normalized, Y)
print(f"Loss: {loss}, Accuracy: {accuracy}")

In [ ]:
# Plot data to visualize the accuracy / loss
# Epoch VS Accuracy
plt.figure(figsize=(13, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuraccy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

# Epoch VS Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

plt.tight_layout()
plt.show()

Phase 2 - Three pre-trained models

ResNet50

In [ ]:
!pip install ssqueezepy
!pip install timm
!pip install pytorch-lightning


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.2/127.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadat

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.models import Model, load_model
from sklearn.model_selection import train_test_split
import cv2
import tensorflow as tf



In [ ]:
data_aug = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, fpath, lbls, bs=32, sz=(224, 224), nch=3, shuf=True, aug=False):
        """
        Initializes the data generator with file paths, labels, and settings.
        :param fpath: List of file paths for images.
        :param lbls: Corresponding labels for the images.
        :param bs: Batch size.
        :param sz: Image size (height, width
        :param nch: Number of channels
        :param shuf: Whether to shuffle data at the end of each epoch.
        :param aug: Whether to apply data augmentation.
        """
        self.fpath = fpath
        self.lbls = lbls
        self.bs = bs
        self.sz = sz
        self.nch = nch
        self.shuf = shuf
        self.aug = aug
        self.on_epoch_end()

    def __len__(self):
        """Returns the number of batches per epoch."""
        return int(np.floor(len(self.fpath) / self.bs))

    def __getitem__(self, idx):
        """
        Generates one batch of data.
        :param idx: Index of the batch.
        :return: Tuple of (batch of images, batch of labels).
        """
        # Get indexes for the batch
        b_idxs = self.idx_arr[idx * self.bs: (idx + 1) * self.bs]
        b_fpaths = [self.fpath[i] for i in b_idxs]
        b_lbls = [self.lbls[i] for i in b_idxs]

        # Generate data
        X, y = self.__data_gen(b_fpaths, b_lbls)
        return X, y


    def on_epoch_end(self):
        """Shuffles the data at the end of each epoch if specified."""
        self.idx_arr = np.arange(len(self.fpath))
        if self.shuf:
            np.random.shuffle(self.idx_arr)

    def __data_gen(self, b_fpaths, b_lbls):
        """
        Generates data for a batch.
        :param b_fpaths: list of file paths for the batch.
        :param b_lbls: labels for the batche
        :return: Tuple of (batch of images, batch of encoded labels)
        """
        X = np.empty((self.bs, *self.sz, self.nch))
        y = np.empty((self.bs), dtype=int)
        for i, (fpath, lbl) in enumerate(zip(b_fpaths, b_lbls)):
            img = np.load(fpath)
            resized_img = np.stack([cv2.resize(ch, self.sz) for ch in img], axis=-1)
            resized_img = np.mean(resized_img, axis=-1, keepdims=True)
            resized_img = np.repeat(resized_img, self.nch, axis=-1)

            if self.aug:
                resized_img = data_aug.random_transform(resized_img)

            X[i] = resized_img
            y[i] = lbl
        return X, tf.keras.utils.to_categorical(y, num_classes=2)


In [ ]:
import os
from sklearn.model_selection import train_test_split

# directories containing images for each class yes and noe
yes_dir = '/content/drive/MyDrive/KaggleDataset/scaleograms/yes'
no_dir = '/content/drive/MyDrive/KaggleDataset/scaleograms/no'

# file paths for each class
yes_files = [os.path.join(yes_dir, f) for f in os.listdir(yes_dir)]
no_files = [os.path.join(no_dir, f) for f in os.listdir(no_dir)]

# Combine file and create labels
fpath = yes_files + no_files
lbls = [1] * len(yes_files) + [0] * len(no_files)

# Split data into training and testing
train_fpaths, test_fpaths, train_lbls, test_lbls = train_test_split(fpath, lbls, test_size=0.2, random_state=42)

# Initialze data generator
batch_size = 32
train_gen = CustomDataGen(train_fpaths, train_lbls, bs=batch_size, aug=True)
test_gen = CustomDataGen(test_fpaths, test_lbls, bs=batch_size, aug=False)


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

# Define and compile the modele
img_rows, img_cols = 224, 224
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))


In [ ]:

# Freeze all layers of the ResNet model to prevent their weights from being updated
for layer in resnet.layers:
    layer.trainable = False

# Define the custom top layers to add on top of the base ResNet50 model
def top(bottom_model, num_classes):
    # Get the output of the base model
    top_model = bottom_model.output

    # Apply global avg pooling to reduce spatial dimensions
    top_model = GlobalAveragePooling2D()(top_model)

    #dropout layer with 50% dropout rate to prevent overfitting
    top_model = Dropout(0.5)(top_model)

    # Add a Dense layer and ReLU activation
    top_model = Dense(1024, activation='relu')(top_model)

    # Add another Dropout layer
    top_model = Dropout(0.5)(top_model)

    # Add another Dense layer with and ReLU activaation
    top_model = Dense(1024, activation='relu')(top_model)

    # another Dropout
    top_model = Dropout(0.5)(top_model)

    # Add Dense layer with and ReLU
    top_model = Dense(512, activation='relu')(top_model)

    # Add the final Dense layer with units equal to the number of classes and softmax activation for classification
    top_model = Dense(num_classes, activation='softmax')(top_model)

    return top_model

# Number of output classes for classification
num_classes = 2

# add the custom top layers to the base ResNet model
top_layer = top(resnet, num_classes)

# create a new model combining the base ResNet and the custom top layers
model = Model(inputs=resnet.input, outputs=top_layer)

# Compile the model with Adam optimizer, categorical cross-entropy loss, and accuracy as the evaluation metric
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:

# Define a learning rate scheduler
def scheduler(epoch, lr):
  # Keep learning rate constant for the first 3 epoch after, decay it exponentially
    if epoch < 3:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

# Early stopping callback to halt training when no improvement in validation loss
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Model checkpoint callback to save the best model based on validation loss
model_check = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)


In [ ]:
# Train the model
history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=10,
    verbose=1,
    callbacks=[early_stop]
)

Resnet50 Gradio


In [ ]:
import gradio as gr
import numpy as np
import cv2
import os
import traceback

# Load the fine-tuned model
mdl = load_model('/content/drive/MyDrive/KaggleDataset/Models/Resnet50_best_model.keras')

def pred(file):
    try:
        # Load and preprocess image data
        data = np.load(file.name)
        if data.ndim == 3:
            # Resize and prepare image
            img = np.stack([cv2.resize(ch, (224, 224)) for ch in data], axis=-1)
            img = np.mean(img, axis=-1, keepdims=True)
            img = np.repeat(img, 3, axis=-1)
            img = np.expand_dims(img, axis=0)

            # Predict and get class label
            pred = mdl.pred(img)
            idx = np.argmax(pred, axis=1)[0]
            conf = pred[0][idx]
            label = ['Healthy', 'Schizophrenia'][idx]

            return label, conf
        else:
            return "Invalid data shape", 0.0

    except Exception as e:
        print("Error during prediction:", str(e))
        traceback.print_exc()
        return "Error during prediction", 0.0
#check error for loading

# Define the Gradio interface
iface = gr.Interface(
    fn=pred,                             # Function to call
    inputs=gr.File(label="Upload .npy File"), # File input
    outputs=[gr.Label(num_top_classes=2), gr.Textbox()], #  label and confidence
    title="Schizophrenia Detection",
    description="Upload a .npy file and get the prediction."
)

# Launch the interface
iface.launch(debug=True)




InceptionV3

In [ ]:
# code here for inceptionv3

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
import gradio as gr
import traceback

In [ ]:
# Directories
yes_dir = '/content/drive/MyDrive/KaggleDataset/scaleograms/yes'
no_dir = '/content/drive/MyDrive/KaggleDataset/scaleograms/no'

# Custom data generator class inheriting from tf.keras.utils.Sequence
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, f_paths, labels, b_size=32, img_size=(299, 299), n_chals=3, shuffle=True):
        # Initialization method
        self.f_paths = f_paths  # file paths
        self.labels = labels  #labels
        self.b_size = b_size  # Batch size
        self.img_size = img_size  # Image size
        self.n_chals = n_chals  # nnumber of channels
        self.shuffle = shuffle  # Shuffle or no
        self.epoch_end()

    def __len__(self):
        # Returns the number of batches per epoch
        return int(np.floor(len(self.f_paths) / self.b_size))

    def __getitem__(self, index):
        # Generate one batch of data
        indxs = self.indxs[index * self.b_size: (index + 1) * self.b_size]
        f_paths_temp = [self.f_paths[k] for k in indxs]  # temp list of file paths for this batch
        labels_temp = [self.labels[k] for k in indxs]  #  labels for this batch

        X, y = self._data_gen(f_paths_temp, labels_temp)  # Gnenerate data
        return X, y

    def epoch_end(self):
        # Updates indices after each epoch
        self.indxs = np.arange(len(self.f_paths))
        if self.shuffle:
            np.random.shuffle(self.indxs)

    def _data_gen(self, f_paths_temp, labels_temp):
        # Generates data containing batch_size samples **migh need to change
        X = np.empty((self.b_size, *self.img_size, self.n_chals))
        y = np.empty((self.b_size), dtype=int)
        for i, (f_paths, label) in enumerate(zip(f_paths_temp, labels_temp)):
            img = np.load(f_paths)
            resized = np.stack([cv2.resize(channel, self.img_size) for channel in img], axis=-1)  # Resize image
            resized = np.mean(resized, axis=-1, keepdims=True)  # Convert to single channel by averaging
            resized = np.repeat(resized, self.n_chals, axis=-1)  # Repeat the single channel to match n_chals
            X[i,] = resized
            y[i] = label
        return X, tf.keras.utils.to_categorical(y, num_classes=2)  # Convert labels to categorical

# lis of file paths for yes and no
yes_files = [os.path.join(yes_dir, f) for f in os.listdir(yes_dir)]
no_files = [os.path.join(no_dir, f) for f in os.listdir(no_dir)]

# Combine filepaths and labels
f_paths = yes_files + no_files
labels = [1] * len(yes_files) + [0] * len(no_files)

# split into training and testing sets
train_paths, test_paths, train_labels, test_labels = train_test_split(f_paths, labels, test_size=0.2, random_state=42)

# batch size
b_size = 32

# create training and testing data generators
train_gen = CustomDataGen(train_paths, train_labels, b_size=b_size)
test_gen = CustomDataGen(test_paths, test_labels, b_size=b_size)

#  set up the InceptionV3 model
img_rows, img_colms = 299, 299
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_rows, img_colms, 3))



87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:

# Freeze the layers of the pre-trained modle
for layer in inception.layers:
    layer.trainable = False

# Define the new top layers to be added to the pre-trained model
def top_(bottom_model, num_classes):
    top_m = bottom_model.output
    top_m = GlobalAveragePooling2D()(top_m)  # Apply global average pooling
    top_m = Dense(1024, activation='relu')(top_m)  # add a dense layer and ReLU activation
    top_m = Dense(1024, activation='relu')(top_m)  # add another dense layer with 1024 units and ReLU activation
    top_m = Dense(512, activation='relu')(top_m)  # ad a dense layer and ReLU activation
    top_m = Dense(num_classes, activation='softmax')(top_m)  # add the final output layer with softmax activation
    return top_m

# Number of classes
num_classes = 2

# Create the new model by combining the pre-trained model and the new top layers
Top = top_(inception, num_classes)
model = Model(inputs=inception.input, outputs=Top)





In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Callback

# Define a learning rate schedler function
def sched(epoch, lr):
    if epoch < 10:
        return lr  # Keep the learning rate the same for the first 10 epochs
    else:
        return lr * 0.1  # Reduce the learning rate by a factor of 10 after epoch 10

# Early stopping callback to stop trainning when validation loss stops improving
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Learning rate scheduler callback
sched = LearningRateScheduler(sched)

# Model checkpoint callback to save the best model based on validation loss
model_cp = ModelCheckpoint('/content/drive/MyDrive/KaggleDataset/Models/InceptionV3_best_model.keras',
                           monitor='val_loss', save_best_only=True)

# compile the model with speciied optimizer, loss function, and metrics
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train model
history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=20,
    verbose=1,
    callbacks=[early_stop, sched, model_cp]
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 496s 17s/step - accuracy: 0.5225 - loss: 3.5130 - val_accuracy: 0.4062 - val_loss: 0.8272 - learning_rate: 0.0010
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 433s 15s/step - accuracy: 0.5461 - loss: 0.6868 - val_accuracy: 0.6094 - val_loss: 0.6549 - learning_rate: 0.0010
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.6175 - loss: 0.6351 

Inceptionv3 Gradio

In [ ]:
# code here for inceptionv3 gradio

In [ ]:
import numpy as np
import gradio as gr
import cv2
import traceback
from tensorflow.keras.models import load_model

model_path = '/content/drive/MyDrive/KaggleDataset/Models/InceptionV3_best_model.keras'
model = load_model(model_path)

def process_and_predict(file):
    try:
        # Load the .npy file
        data = np.load(file.name)

        if data.ndim == 3:
            # Resize and preprocess the image
            resized = np.stack([cv2.resize(channel, (299, 299)) for channel in data], axis=-1)
            resized = np.mean(resized, axis=-1, keepdims=True)
            resized = np.repeat(resized, 3, axis=-1)
            resized = np.expand_dims(resized, axis=0)

            # Make prediction
            pred = model.predict(resized)
            idx = np.argmax(pred, axis=1)[0]
            confidence = pred[0][idx]
            class_label = ['Healthy', 'Schizophrenia']

            return class_label[idx], float(confidence)
        else:
            return "Invalid data shape", 0.0

    except Exception as e:
        print("Error during prediction:", str(e))
        traceback.print_exc()
        return "Error during prediction", 0.0

# Define the Gradio interface
iface = gr.Interface(
    fn=process_and_predict,
    inputs=gr.File(label="Upload .npy File"),
    outputs=[gr.Label(num_top_classes=2), gr.Textbox()],
    title="Schizophrenia Detection"
)

# Launch the interface
iface.launch(debug=True)



VGG16

In [ ]:
def get_npy_files(directory):
  return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.npy')]

#Define paths to scaleograms
yes_dir = '/content/drive/MyDrive/KaggleDataset/scaleograms/yes'
no_dir = '/content/drive/MyDrive/KaggleDataset/scaleograms/no'

yes_files = get_npy_files(yes_dir)
no_files = get_npy_files(no_dir)

#Custom data generator
class CustomDataGen(tf.keras.utils.Sequence):
  def __init__(self, file_paths, labels, img_size=(224, 224), n_channels=3, batch_size=32, shuffle=True, repeat_factor=30, **kwargs):
    self.original_file_paths = file_paths
    self.original_labels = labels
    self.batch_size = batch_size
    self.img_size = img_size
    self.n_channels = n_channels
    self.shuffle = shuffle
    self.repeat_factor = repeat_factor
    self.file_paths, self.labels = self.repeat_data()
    self.on_epoch_end()
    super(CustomDataGen, self).__init__(**kwargs)

  def repeat_data(self):
    file_paths_repeated = self.original_file_paths * self.repeat_factor
    labels_repeated = self.original_labels * self.repeat_factor
    return file_paths_repeated, labels_repeated

  def __len__(self):
    return int(np.floor(len(self.file_paths) / self.batch_size))

  def __getitem__(self, index):
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
    file_paths_temp = [self.file_paths[k] for k in indexes]
    labels_temp = [self.labels[k] for k in indexes]
    X, y = self.__data_generation(file_paths_temp, labels_temp)
    return X, y

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.file_paths))
    if self.shuffle:
      np.random.shuffle(self.indexes)

  def __data_generation(self, file_paths_temp, labels_temp):
    X = np.empty((self.batch_size, *self.img_size, self.n_channels))
    y = np.empty((self.batch_size), dtype=int)

    for i, (file_path, label) in enumerate(zip(file_paths_temp, labels_temp)):
       img = np.load(file_path)
       resized_img = np.stack([cv2.resize(channel, self.img_size) for channel in img], axis=-1)

       if resized_img.shape[-1] != self.n_channels:
        resized_img = np.mean(resized_img, axis=-1, keepdims=True)
        resized_img = np.repeat(resized_img, self.n_channels, axis=-1)

       X[i,] = resized_img
       y[i] = label

    return X, to_categorical(y, num_classes=2)

oversample_no_files = random.choices(no_files, k=len(yes_files))
file_paths = yes_files + oversample_no_files
labels = [1] * len(yes_files) + [0] * len(oversample_no_files)

combined = list(zip(file_paths, labels))
random.shuffle(combined)
file_paths, labels = zip(*combined)
file_paths = list(file_paths)
labels = list(labels)

train_paths, test_paths, train_labels, test_labels = train_test_split(file_paths, labels, test_size=0.2, random_state=42)


train_generator = CustomDataGen(train_paths, train_labels)
validation_generator = CustomDataGen(test_paths, test_labels)


#Load VGG16 Model
img_rows, img_cols = 224, 224
vgg = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))





In [ ]:
#Add custom layers on top of VGG16

def add_custom_layers_with_dropout(base_model, num_classes):
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
  x = Dropout(0.5)(x)
  x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
  x = Dropout(0.5)(x)
  x = Dense(num_classes, activation='softmax')(x)
  return x

num_classes = 2
custom_output = add_custom_layers_with_dropout(vgg, num_classes)
model_with_dropout = Model(inputs=vgg.input, outputs=custom_output)




In [ ]:
#Phase 1: Train the custom layers with frozen base model layers

for layer in vgg.layers:
  layer.trainable = False

model_with_dropout.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



checkpoint_phase1 = ModelCheckpoint('best_model_phase1.keras', monitor='val_loss', mode='min', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=1e-6)

history_phase1 = model_with_dropout.fit(train_generator, steps_per_epoch=len(train_paths) // 32, validation_data=validation_generator, validation_steps =len(test_paths) // 32, epochs=10, verbose=1, callbacks=[checkpoint_phase1, early_stopping, reduce_lr])

Epoch 1/10


In [ ]:
#Phase 2: Unfreeze some layers in base network and jointly train
for layer in vgg.layers[-4:]:
  layer.trainable = True

model_with_dropout.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint_phase2 = tf.keras.callbacks.ModelCheckpoint('best_model_finetune.keras', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

history_phase2 = model_with_dropout.fit(train_generator, steps_per_epoch=len(train_paths) // 32, validation_data=validation_generator, validation_steps=len(test_paths) // 32, epochs=20, verbose=1, callbacks=[checkpoint_phase2, early_stopping, reduce_lr])



In [ ]:
#Plot training and validation accuracy and loss

def plot_training_history(history, phase):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(acc))

  plt.figure()
  plt.plot(epochs, acc, 'r', label='Training accuracy')
  plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
  plt.title('Training and validation accuracy - {phase 1}')
  plt.legend(loc=0)

  plt.figure()
  plt.plot(epochs, loss, 'r', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss - {phase 2}')
  plt.legend(loc=0)

  plt.show()

plot_training_history(history_phase1, 'Phase 1')
plot_training_history(history_phase2, 'Phase 2')

model_with_dropout.save('best_model_finetune.keras')

best_model = load_model('best_model_finetune.keras')

vallidation_generator = CustomDataGen(test_paths, test_labels, img_size=(224, 224), n_channels=3, batch_size=32, shuffle=False)

loss, accuracy = best_model.evaluate(validation_generator)
print('Test accuracy:', accuracy)
print('Test Loss: ', loss)

VGG16 Gradio

In [ ]:
#Load the fine-tuned model
model = load_model('/content/drive/MyDrive/KaggleDataset/best_model_finetune.keras')

def process_and_predict(file):
  try:
    data = np.load(file.name)

    if data.ndim == 3:
      resized_img = np.stack([cv2.resize(channel, (224, 224)) for channel in data], axis=-1)
      resized_img = np.mean(resized_img, axis=-1, keepdims=True)
      resized_img = np.repeat(resized_img, 3, axis=-1)
      resized_img = np.expand_dims(resized_img, axis=0)


      prediction = model.predict(resized_img)
      class_idx = np.argmax(prediction, axis=1)[0]
      confidence = prediction[0][class_idx]
      class_label = ['Healthy', 'Schizophrenia'][class_idx]

      return class_label, confidence
    else:
      return "Invalid data shape", 0.0

  except Exception as e:
    print("Error during prediction:", str(e))
    traceback.print_exc()
    return "Error during prediction", 0.0

iface = gr.Interface(
  fn=process_and_predict,
  inputs=gr.File(label="Upload .npy File"),
  outputs=[gr.Label(num_top_classes=2), gr.Textbox()],
  title="Schizophrenia Detection",
  description="Upload a .npy file and get the prediction."
)

iface.launch(debug=True)





Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://353eeb37a1fa20abbc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://353eeb37a1fa20abbc.gradio.live


Ensemble Model + Gradio

In [ ]:
def load_and_preprocess_npy_files(file_paths, img_size):
  data = []
  labels = []
  for file_path in file_paths:
    label = 1 if 'yes' in file_path else 0
    img = np.load(file_path)
    resized_img = np.stack([cv2.resize(channel, img_size) for channel in img], axis=-1)
    if resized_img.shape[-1] != 3:
      resized_img = np.mean(resized_img, axis=-1, keepdims=True)
      resized_img = np.repeat(resized_img, 3, axis=-1)
    data.append(resized_img)
    labels.append(label)

  # Convert lists to numpy array
  data = np.array(data)
  labels = np.array(labels)
  return data, labels

  #Directories for 'yes' and 'no' scaleograms

yes_dir = '/content/drive/MyDrive/KaggleDataset/scaleograms/yes'
no_dir = '/content/drive/MyDrive/KaggleDataset/scaleograms/no'

yes_files = [os.path.join(yes_dir, f) for f in os.listdir(yes_dir) if f.endswith('.npy')]
no_files = [os.path.join(no_dir, f) for f in os.listdir(no_dir) if f.endswith('.npy')]


#Oversample the 'no' class to balance the dataset
oversampled_no_files = random.choices(no_files, k=len(yes_files))
all_files = yes_files + oversampled_no_files
labels = [1] * len(yes_files) + [0] * len(oversampled_no_files)

#Shuffle the combined dataset
combined = list(zip(all_files, labels))
random.shuffle(combined)
all_files, labels = zip(*combined)
all_files = list(all_files)
labels = list(labels)

#Split the dataset into training and testing sets
train_paths, test_paths, train_labels, test_labels = train_test_split(all_files, labels, test_size=0.2, random_state=42)

# Load the pre-trained models
vgg16_model = load_model('/content/drive/MyDrive/KaggleDataset/best_model_finetune.keras')
resnet50_model = load_model('/content/drive/MyDrive/KaggleDataset/Resnet50_best_model.keras')
inceptionv3_model = load_model('/content/drive/MyDrive/KaggleDataset/InceptionV3_best_model.keras')

def preprocess_image_for_model(img, model_name):
  if model_name == 'inceptionv3':
    img = cv2.resize(img, (299, 299))
  else:
      img_size = (224, 224)
  resized_img = np.stack([cv2.resize(channel, img_size) for channel in img], axis=-1)
  if resized_img.shape[-1] != 3:
    resized_img = np.mean(resized_img, axis=-1, keepdims=True)
    resized_img = np.repeat(resized_img, 3, axis=-1)
  return resized_img

# Preprocess test data for each model
X_test_vgg16, y_test = load_and_preprocess_npy_files(test_paths, (224, 224))
x_test_resnet50, _ = load_and_preprocess_npy_files(test_paths, (224, 224))
x_test_inceptionv3, _ = load_and_preprocess_npy_files(test_paths, (299, 299))

# Get predictions from each model
vgg16_preds = vgg16_model.predict(X_test_vgg16)
resnet50_preds = resnet50_model.predict(x_test_resnet50)
inceptionv3_preds = inceptionv3_model.predict(x_test_inceptionv3)

#Majority voting for ensemble prediction
def majority_voting(preds):
  preds = np.array(preds)
  majority_voting = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=preds)
  return majority_voting

ensemble_preds = majority_voting([np.argmax(vgg16_preds, axis=1), np.argmax(resnet50_preds, axis=1), np.argmax(inceptionv3_preds, axis=1)])
accuracy = np.mean(ensemble_preds == y_test)

def process_and_predict(file):
  try:
    data = np.load(file.name)
    #Check if data has 3 dimensions
    if data.ndim == 3:
      resized_img_vgg16 = preprocess_image_for_model(data, 'vgg16')
      resized_img_resnet50 = preprocess_image_for_model(data, 'resnet50')
      resized_img_inceptionv3 = preprocess_image_for_model(data, 'inceptionv3')

      #Get predictions from each model
      vgg16_pred = vgg16_model.predict(np.expand_dims(resized_img_vgg16, axis=0))
      resnet50_pred = resnet50_model.predict(np.expand_dims(resized_img_resnet50, axis=0))
      inceptionv3_pred = inceptionv3_model.predict(np.expand_dims(resized_img_inceptionv3, axis=0))

      majority_vote = majority_voting([np.argmax(vgg16_pred, axis=1), np.argmax(resnet50_pred, axis=1), np.argmax(inceptionv3_pred, axis=1)])
      class_idx = majority_vote[0]
      confidence = max(vgg16_pred[0][class_idx], resnet50_pred[0][class_idx], inceptionv3_pred[0][class_idx])
      class_label = ['Healthy', 'Schizophrenia'][class_idx]

      return class_label, confidence
    else:
      return "Invalid data shape", 0.0

  except Exception as e:
    print("Error during prediction:", str(e))
    traceback.print_exc()
    return "Error during prediction", 0.0

iface = gr.Interface(
    fn=process_and_predict,
    inputs=gr.File(label="Upload .npy file"),
    outputs=[gr.Label(num_top_classes=2), gr.Textbox()],
    title="Schizophrenia Detection Ensemble Model",
    description="Upload a .npy file for prediction."
)

iface.launch(debug=True)





KeyboardInterrupt: 